# Databases
## Session 3 - Unit 3 [Self Guided]

<img src=media/databases.jpg width=200/>

Welcome to the "Databases" unit of the Python Academy! In this unit, you will learn:
  - Databases
  - SQL
  - Use Case with SQLite

## Databases

A [**database**](https://en.wikipedia.org/wiki/Database) is a **organized collection of data**. In particular, we are interested in databases supported by a database-management system (**DBMS**), think MySQL or Oracle. DBMS provide an interface to perform various management operations, like creating databases, storing and updating data, creating tables, managing users and a lot more. When we talk about databases, we are often referring to the combination of the actual database and the corresponding DBMS.

### Relational vs Non-Relational Databases

In a **relational database**, the data is modelled as *rows and columns* in a series of tables and relationships between them. The vast majority uses **SQL** (Structured Query Language) for reading and writing data and, as such, they are also known as SQL databases.

On the other side we have **non-relational databases** that may vary in how they store data (e.g. key-value, document, graph). These are also referred as **NoSQL** because they are based in other query languages. Key advantages include that they are **often fast** and **do not require fixed table schemas**. The choice for a particular NoSQL is much **more dependent on the problem** it must solve. The **data structures** used by NoSQL are different from relational databases and include a wide variety of structures (e.g. key-value, document, graph, wide column).

Since the APIs are much more specific for NoSQL applications, we will present a SQL-based use case instead to showcase managing databases with Python. If you want to experiment with NoSQL on your own, you can try [Redis (key-value cache)](https://docs.redis.com/latest/rs/references/client_references/client_python/), [Neo4j (graph)](https://neo4j.com/developer/python/) or [MongoDB (document)](https://www.mongodb.com/languages/python).

## SQL

To help our introduction to SQL, let's imagine we are building a database **SportsDb** for an electronic sports game, like [NBA 2K](https://en.wikipedia.org/wiki/NBA_2K) or [EA Sports FIFA](https://en.wikipedia.org/wiki/FIFA_(video_game_series). Our goal is to leverage a SQL database to **store and interact with our data**.

The SQL **syntax may differ** between multiple DBMS (e.g. MysQL, PostgreSQL), so our goal is to get the gist of how SQL works in broad terms. For specific details, refer to the documentation of the specific DMBS.

*This is a **(very) simplified overview of SQL**, just to enable you to use it with the other contents this workshop. If you will use SQL on a frequent basis, we (heavily) advise you to learn it on a deeper level.*

### Tables

All right, so we already know we want to build a database for a sports game. So, what will we need? Sports are made of players, teams and venues. To start, our **first challenge** is to **model the *player* entity**. Each player can be described by the *name*, *date of birth*, *nationality*, *playing position*, *speed*, *strength*, *active/retired* properties.

To store our players, we will need to create a **table**, called `players` (innovative name, right?). It's common practice to use singular nouns as table names.

The table is the basic building block of a SQL Database; it is composed by:
  - **rows**, aka records, that aggregate a single entry of data for the entity being modeled
  - **columns**, aka fields, that are the properties characterizing the entity.

### Data Types

Similar to the "Data Types" unit we saw earlier, each column in SQL has an associated data type (all values belong to the same data type).

| Data Type   | Description                                           | Python Equivalent                     |
|-------------|-------------------------------------------------------|---------------------------------------|
| INTEGER     | Integer numeric                                       | `int`                                 |
| FLOAT       | decimal numeric                                       | `float`                               |
| VARCHAR(n)  | variable length character string, up to n characters. | `str`                                 |
| DATE        | year, month, day                                      | `datetime.date`                       |
| TIMESTAMP   | year, month, day, hour, minute, second, millisecond   | `datetime.datetime`, `pd.Timestamp`   |
| BOOLEAN     | true or false                                         | `bool`                                |

### Exercise - Table Data Types

What is the data type we should assign to each column of our *players* table?

*name*, *date of birth*, *nationality*, *playing position*, *speed*, *strength*, *active/retired* 

### Contraints

We can define **constraints** on the values to **limit the type of data that can be stored in a table**. These rules determine if a certain value may belong to a column or not. This ensures accuracy and reliability of data; any action that violates the constraints is aborted. 

Constraints can be applicable at **column-level** (limit column data only) or at **table-level** (limit whole table).

We advise you to check the full list of [SQL Constraints](https://www.studytonight.com/dbms/sql-constraints.php) at ease afterwards this unit. For now, we will highlight some of the most common and useful constraints.

| Constraint                | Description                                                                                       |
| ------------------------- | ------------------------------------------------------------------------------------------------- |
| NOT NULL                  | column may not have NULL values (i.e. missing)                                                    |
| UNIQUE                    | column values may not be duplicates                                                               |
| DEFAULT                   | when inserting records, if no value is specified for the column, the default value is used        |
| CHECK                     | restrict column values between a range                                                            |



### Index, Keys

Remember how we use indices as key elements to retrieve data in a faster way with `pandas.DataFrame`? The same concept applies to SQL databases.

Using **indexed columns** is advised for tables that are consulted frequently. On the flipside, indexing slows data operations (insert, delete, update), as such tables need to be reindexed when the data changes.

---

Besides the indices, **keys** are used to identify records in a table and create relationship between tables.
  - A **primary key** uniquely identifies each record in a table. By definition, it must be NOT NULL and UNIQUE, otherwise we couldn't uniquely identify the records. It can be a single column or a combination of multiple columns.
  - A **foreign key** references the primary key in another table, working as a mapping on how to relate two tables.

## Practical Use Case

Let's get our hands dirty! We will build our SportsDB database from scratch and start loading some data to get our project up and running.

To speed things up, we will use [SQLite](https://docs.python.org/3/library/sqlite3.html). Using a syntax similar to SQL, **SQLite**
is a **lightweight**, **self-contained**, disk-based database that doesn't require a separate server process. This makes it very useful for **very small projects**, and education purposes like this class!

We will use `pandas` package to perform generic data writing and loading tasks, and `sqlite3` to handle QSLite specific tasks. Note: we are just doing this now, but **imports should always be put at the top of the file!** Don't just take our word for it, it is actually a [PEP8 convention](https://peps.python.org/pep-0008/#imports) to do it so!

In [1]:
import sqlite3
import pandas as pd

To create a database, we first establish a [Connection](https://docs.python.org/3/library/sqlite3.html#connection-objects) that represents the database. Afterwards, from the connection we get a [Cursor](https://docs.python.org/3/library/sqlite3.html#cursor-objects) to execute our commands. 

*Don't worry too much about theses details for now, just keep in mind the main objective we want to achieve.*

In [2]:
# establish a connection. creates database if doesn't exist
con = sqlite3.connect('data/SportsDB/SportsDB.db')

# get the cursor
cur = con.cursor()

We already have some players data in a `players.csv` to start building our table in the database. We will load the data into pandas, get a sense of the data, and create the table afterwards.


In [3]:
players_df = pd.read_csv('data/SportsDB/players.csv')
players_df.head()

,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,Nikola Jokic,19-02-1995,SRB,C,63,93,False
1,Bruce Bowen,14-06-1971,USA,SF/SG,83,85,True
2,Giannis Antetokounmpo,06-12-1994,GRC,PF/C,90,94,False
3,Dirk Nowitzki,19-06-1978,DEU,PF/C,65,82,True
4,Manu Ginobili,28-07-1977,ARG,SG/SF,92,80,True


Awesome! Since we already know some things about SQL, let's create a new `players` table, considering all the properties we mentioned before (data types, constraints, indices, keys).

In [4]:
# since we are re-running stuff multiple times in the notebook, better to have this to clean the table 
cur.execute('DROP TABLE IF EXISTS players')

In [5]:
try:
    cur.execute('''CREATE TABLE players (
                    ID INTEGER PRIMARY KEY AUTOINCREMENT,
                    NAME TEXT NOT NULL,
                    DOB TEXT NOT NULL,
                    NATIONALITY CHAR(3),
                    POSITION VARCHAR(5),
                    SPEED INT,
                    STRENGTH INT,
                    RETIRED INT CHECK (RETIRED IN (0, 1))
                );
            ''')
except Exception as e:
    print(e)

Remember how we talked about the general idea of SQL and that each DMBS has its own specific implementation details? Well, SQLite has a couple of *[quirks](https://sqlite.org/quirks.html)* which you may not expect, including:
  - SQLite doesn't support **native timestamp or date** data types (more on [docs](https://sqlite.org/lang_datefunc.html)).
  - SQLite doesn't support a **native boolean** data type. Instead, we consider an INT taking the value of 0 or 1.


There are somethings we haven't talked about yet:
  1. What is the **AUTOINCREMENT** doing in the ID column?
  2. What is the difference between **CHAR and VARCHAR**?
  3. Why is NATIONALITY limited to 3 characters? [*hint*](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3)

The table is created. Let's check its records.

In [6]:
# all right! we have an empty table
cur.execute("SELECT * FROM players").fetchall()

[]

Using the SQLite API, the **cursor executes SQLite statements and fetches data from queries results**. We can `.fetchone()`, `.fetchmany(size)` or `.fetchall()`. The drawback is that you will need to preprocess the data before you are able to use it to perform data analysis or calculate properties at a group/tabular level.

---

To add data into our database, Pandas enables you to **write from a DataFrame directly to a SQL database** with `.to_sql` method ([API reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html)). We had to change a couple of the parameters from their defaults for this to run.

In [7]:
players_df.to_sql("players", con=con, if_exists='append', index=False)

5

Awesome! Your new table should be ready by now! Let's take a look and see if everything went right.

In [8]:
# now we have some records!
cur.execute("SELECT * FROM players").fetchall()

[(1, 'Nikola Jokic', '19-02-1995', 'SRB', 'C', 63, 93, 0),
 (2, 'Bruce Bowen', '14-06-1971', 'USA', 'SF/SG', 83, 85, 1),
 (3, 'Giannis Antetokounmpo', '06-12-1994', 'GRC', 'PF/C', 90, 94, 0),
 (4, 'Dirk Nowitzki', '19-06-1978', 'DEU', 'PF/C', 65, 82, 1),
 (5, 'Manu Ginobili', '28-07-1977', 'ARG', 'SG/SF', 92, 80, 1)]

Alternatively to the SQLite API, you can use pandas `.read_sql` to import the query results directly into a DataFrame and allow you to process data much more efficiently afterwards (in comparison to sqlite api)

In [9]:
# we can also run SQL queries into a DataFrame
pd.read_sql("SELECT * from players", con=con)

,ID,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,1,Nikola Jokic,19-02-1995,SRB,C,63,93,0
1,2,Bruce Bowen,14-06-1971,USA,SF/SG,83,85,1
2,3,Giannis Antetokounmpo,06-12-1994,GRC,PF/C,90,94,0
3,4,Dirk Nowitzki,19-06-1978,DEU,PF/C,65,82,1
4,5,Manu Ginobili,28-07-1977,ARG,SG/SF,92,80,1


## SQL Queries

A **query** is the operation of **retrieving data from a database**. Most of the SQL syntax leverages keywords that resemble English phrases, in such a way that reading a SQL query is (almost) self-explanatory of the operation you want to achieve.

The majority of the DBMS provide a way for you to run queries, either graphically (UI) or programatically. For a Python approach, we will continue to use `pandas` as the interface that is 1. able to run SQL queries; and 2. retrieve results into a nice easy-to-use DataFrame.

### SELECT

**SELECT** is the basic instruction to **retrieve data**. `*` is a wildcard that in conjunction with SELECT means to retrieve data from all columns in a table. Alternatively, you can specify specific columns to be retrieved.

In [10]:
# retrieve all colums
pd.read_sql('SELECT * FROM players', con=con)

,ID,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,1,Nikola Jokic,19-02-1995,SRB,C,63,93,0
1,2,Bruce Bowen,14-06-1971,USA,SF/SG,83,85,1
2,3,Giannis Antetokounmpo,06-12-1994,GRC,PF/C,90,94,0
3,4,Dirk Nowitzki,19-06-1978,DEU,PF/C,65,82,1
4,5,Manu Ginobili,28-07-1977,ARG,SG/SF,92,80,1


In [11]:
# select NAME, DOB columns
pd.read_sql('SELECT NAME, DOB FROM players', con=con)

,NAME,DOB
0,Nikola Jokic,19-02-1995
1,Bruce Bowen,14-06-1971
2,Giannis Antetokounmpo,06-12-1994
3,Dirk Nowitzki,19-06-1978
4,Manu Ginobili,28-07-1977


### WHERE

The conditioning for retrieving data is made available with **WHERE**. You can specify single conditions, or use AND/OR logical operators to combine WHERE conditions.

In [12]:
# retired players
pd.read_sql('SELECT * FROM players WHERE RETIRED = 1', con=con)

,ID,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,2,Bruce Bowen,14-06-1971,USA,SF/SG,83,85,1
1,4,Dirk Nowitzki,19-06-1978,DEU,PF/C,65,82,1
2,5,Manu Ginobili,28-07-1977,ARG,SG/SF,92,80,1


In [13]:
# retired, german players
pd.read_sql('SELECT * FROM players WHERE RETIRED = 1 AND NATIONALITY = "DEU"', con=con)

,ID,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,4,Dirk Nowitzki,19-06-1978,DEU,PF/C,65,82,1


### LIMIT

By default, the queries will retrieve **all records** from the table (which is not always a great thing). To limit the number of retrievable rows, we use the **LIMIT**.

In [14]:
# get 3 players only
pd.read_sql('SELECT * FROM players LIMIT 3', con=con)

,ID,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,1,Nikola Jokic,19-02-1995,SRB,C,63,93,0
1,2,Bruce Bowen,14-06-1971,USA,SF/SG,83,85,1
2,3,Giannis Antetokounmpo,06-12-1994,GRC,PF/C,90,94,0


### ORDER BY

In many analysis, it matters the order by which we retrieve data. As such, we tell SQL to **ORDER BY** some attributes. The **ASC**ending is the default, so it can be omitted; but the **DESC** must be specified for descending order.

In [15]:
# top 3 strength
pd.read_sql('SELECT * FROM players ORDER BY STRENGTH DESC LIMIT 3', con=con)

,ID,NAME,DOB,NATIONALITY,POSITION,SPEED,STRENGTH,RETIRED
0,3,Giannis Antetokounmpo,06-12-1994,GRC,PF/C,90,94,0
1,1,Nikola Jokic,19-02-1995,SRB,C,63,93,0
2,2,Bruce Bowen,14-06-1971,USA,SF/SG,83,85,1


### GROUP BY and aggregates

We often want to analyze data **per a specific entity** (aka group), instead of at a row (aka record) level. The GROUP BY joins the records that have the same values for the field(s) we specify and we can perform aggregate calculations on top of such data (e.g. COUNT, AVG, MIN, MAX) 

In [16]:
# GROUP BY - With HAVING
pd.read_sql('''
            SELECT 
                RETIRED,
                AVG(STRENGTH) AS avg_stg,
                MAX(SPEED)    AS max_speed
            FROM players
            GROUP BY RETIRED
            ''', con=con)

,RETIRED,avg_stg,max_speed
0,0,93.500000,90
1,1,82.333333,92


See how we split our query into multiple lines for an easier reading?

### HAVING

The same way WHERE is used to filter rows, HAVING filters data at a group level.

In [17]:
# GROUP BY - With HAVING
pd.read_sql('''
            SELECT 
                RETIRED,
                AVG(STRENGTH) AS avg_stg,
                MAX(SPEED)    AS max_speed
            FROM players
            GROUP BY RETIRED
            HAVING max_speed > 91
            ''', con=con)

,RETIRED,avg_stg,max_speed
0,1,82.333333,92


## Databases vs Files

We've seen how we can use Files in the previous unit to read and store data. Now, the Databases seem like they are also built for reading and writing data. Is one better than the other? When should we use Files and when should we use Databases?

Typically, a **Database** is used for related and structured data. Databases are useful when you need to **relate tables**, perform **fast lookups/sorts** by more than one value (leveraging indexes) and perform **read/write** operations by **multiple users** at the same time. 

**Files** are more appropriate for **unstructed**, **arbitrary or unrelated data**, when you have chunks of data that grow frequently, need other applications to access your data without a specific API or want to use version control on your data.

In short, the answer is ***it depends***. Understand your's and your team's need first, then you will be able to know whether Files or Databases are more appropriate.

## Recap

Congratulations, you made it all the way the "Databases" unit! By the end of this notebook, you should have a clear idea of:
  1.  What are **Databases**;
  2.  **Relational and Non-Relational** Databases;
  3.  How to bootstrap a project with **SQLite**;
  4.  SQL **Queries** and their purpose;
  5.  How to decide between **Databases or Files**.